In [1]:
from GvaDataset import GvaDataset
from torch.utils.data import random_split, DataLoader
import torchvision.transforms as transforms 
import torch

transform = transforms.Compose([
    transforms.Resize((1024, 512)),
    transforms.RandomHorizontalFlip(p=0.5)
])

images_path = "/projectnb/dunlop/chrisdc/gva_senior_design/image_processing/data/_Original"
labels_path = "/projectnb/dunlop/chrisdc/gva_senior_design/image_processing/data/labels"
dataset = GvaDataset(images_path, labels_path, transform=transform)

total_size = len(dataset)
train_size = int(total_size * 0.8)  # 80% for training
test_size = total_size - train_size  # 20% for testing

train_dataset = torch.utils.data.Subset(dataset, range(train_size))
test_dataset = torch.utils.data.Subset(dataset, range(train_size, train_size+test_size))

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [6]:
import torch
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm


class Experiment:
    def __init__(self, model, criterion, optimizer, device='cpu', starting_epoch=0):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device
        self.starting_epoch = starting_epoch

        self.validation_loss_history = []
        self.train_loss_history = []

    def train(self, train_loader, validation_loader, epochs=10):
        N = len(train_loader)
        for epoch in range(1, epochs + 1):
            self.model.train()
            epoch_loss = 0
            with tqdm(enumerate(train_loader), 
                      total=N, 
                      desc=f'Epoch {self.starting_epoch + epoch}/{self.starting_epoch + epochs}', 
                      unit='batch') as pbar:
                for i, data in pbar:
                    images, labels, _ = data

                    images = images.to(self.device)
                    labels = labels.to(self.device)

                    self.optimizer.zero_grad()

                    pred = self.model(images)

                    pred = torch.squeeze(pred)
                    labels = torch.squeeze(labels)
                
                    loss = self.criterion(pred, labels)
                    loss.backward()
                    self.optimizer.step()

                    epoch_loss += loss.item()
                    pbar.set_postfix(loss = loss.item(), avg_loss=epoch_loss / (i + 1))

            self.model.eval()
            with torch.no_grad():
                epoch_val_loss = 0
                for images, labels, _ in validation_loader:
                    images = images.to(self.device)
                    labels = labels.to(self.device)
                    pred = self.model(images)
                    labels = torch.squeeze(labels)
                    pred = torch.squeeze(pred)
                    val_loss = self.criterion(pred, labels)
                    epoch_val_loss += val_loss.item()

            print(f'{self.starting_epoch +  epoch}/{self.starting_epoch + epochs} train loss: {epoch_loss / N}  val loss: {epoch_val_loss / len(validation_loader)}')
            self.train_loss_history.append(epoch_loss / N)
            self.validation_loss_history.append(epoch_val_loss / len(validation_loader))
            epoch_loss = 0
        
        self.starting_epoch += epochs
        print("Training Complete")

    def save_weights(self, file_name):
        assert file_name.endswith(".pth")

        torch.save(self.model.state_dict(), file_name)

    def save_experiment(self, file_name):
        assert file_name.endswith(".pkl")

        exp_dict = dict(
            train_loss_history=self.train_loss_history, 
            val_loss_history=self.validation_loss_history, 
            state_dict=self.model.state_dict(),
            starting_epoch = self.starting_epoch
        )

        with open(file_name, 'wb') as f:
            pickle.dump(exp_dict, f)

    def plot_loss(self):
        plt.plot(self.train_loss_history, color='blue', label='Training Loss')
        plt.plot(self.validation_loss_history, color='green', label='Validation Loss')
        plt.title("Training Loss")
        plt.legend()
        plt.xlabel("Epochs")
        plt.ylabel(self.criterion.__name__)

    def load_experiment(self, file_name):

        with open(file_name, 'rb') as f:
            exp_dict = pickle.load(f)
        
        state_dict = exp_dict['state_dict']
        self.model.load_state_dict(state_dict)
        self.starting_epoch = exp_dict['starting_epoch']
        self.val_loss_history = exp_dict['val_loss_history']
        self.train_loss_history = exp_dict['train_loss_history']


In [7]:
from models.unet import UNet
# from experiment import Experiment
from losses import dice_loss
import torch.optim as optim
import torch.nn as nn
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = UNet(in_channels=3, out_channels=2).to(device)
optimizer = optim.Adam(model.parameters())
criterion = dice_loss
experiment = Experiment(model, criterion, optimizer, device)

In [8]:
experiment.train(train_loader=train_loader, validation_loader=test_loader, epochs=5)

Epoch 1/5: 100%|███| 6/6 [00:05<00:00,  1.07batch/s, avg_loss=0.876, loss=0.944]


1/5 train loss: 0.8762735823790232  val loss: 0.9716331362724304


Epoch 2/5: 100%|███| 6/6 [00:05<00:00,  1.09batch/s, avg_loss=0.844, loss=0.705]


2/5 train loss: 0.8442181249459585  val loss: 0.9709388514359792


Epoch 3/5: 100%|███| 6/6 [00:05<00:00,  1.08batch/s, avg_loss=0.843, loss=0.766]


3/5 train loss: 0.8426059285799662  val loss: 0.9726859430472056


Epoch 4/5: 100%|███| 6/6 [00:05<00:00,  1.09batch/s, avg_loss=0.851, loss=0.911]


4/5 train loss: 0.8505076368649801  val loss: 0.9724708298842112


Epoch 5/5: 100%|███| 6/6 [00:05<00:00,  1.09batch/s, avg_loss=0.836, loss=0.842]


5/5 train loss: 0.8355555435021719  val loss: 0.9772857129573822
Training Complete


In [12]:
import os
weights_dir = "/projectnb/dunlop/chrisdc/gva_senior_design/image_processing/weights/"
name = '3-29-24_epochs5'
weights_path = os.path.join(weights_dir, name + '.pkl')
experiment.save_experiment(weights_path)
experiment.load_experiment(weights_path)

In [14]:
experiment.train(train_loader, test_loader, epochs=995)

Epoch 6/1000: 100%|█| 6/6 [00:05<00:00,  1.07batch/s, avg_loss=0.814, loss=0.763


6/1000 train loss: 0.8136072556177775  val loss: 0.9711140791575114


Epoch 7/1000: 100%|█| 6/6 [00:05<00:00,  1.07batch/s, avg_loss=0.813, loss=0.782


7/1000 train loss: 0.8126545151074728  val loss: 0.9836932222048441


Epoch 8/1000: 100%|█| 6/6 [00:05<00:00,  1.07batch/s, avg_loss=0.82, loss=0.914]


...

998/1000 train loss: 0.5978120565414429  val loss: 0.5482129255930582


Epoch 999/1000: 100%|█| 6/6 [00:05<00:00,  1.03batch/s, avg_loss=0.573, loss=0.4


999/1000 train loss: 0.5732827583948771  val loss: 0.8271821935971578


Epoch 1000/1000: 100%|█| 6/6 [00:05<00:00,  1.02batch/s, avg_loss=0.529, loss=0.


1000/1000 train loss: 0.5290567676226298  val loss: 0.7726606329282125
Training Complete


In [15]:
import os
weights_dir = "/projectnb/dunlop/chrisdc/gva_senior_design/image_processing/weights/"
name = '3-29-24_epochs1000'
weights_path = os.path.join(weights_dir, name + '.pkl')
experiment.save_experiment(weights_path)

In [ ]:
experiment.train(train_loader, test_loader, epochs=1000)

Epoch 1001/2000: 100%|█| 6/6 [00:05<00:00,  1.03batch/s, avg_loss=0.637, loss=0.


1001/2000 train loss: 0.636970450480779  val loss: 0.8218806982040405


Epoch 1002/2000: 100%|█| 6/6 [00:05<00:00,  1.05batch/s, avg_loss=0.579, loss=0.


1002/2000 train loss: 0.578954150279363  val loss: 0.7202867070833842


Epoch 1003/2000: 100%|█| 6/6 [00:05<00:00,  1.04batch/s, avg_loss=0.65, loss=0.7


...

Epoch 1660/2000: 100%|█| 6/6 [00:05<00:00,  1.05batch/s, avg_loss=0.447, loss=0.


1660/2000 train loss: 0.4473743438720703  val loss: 0.8764406144618988


Epoch 1661/2000: 100%|█| 6/6 [00:05<00:00,  1.05batch/s, avg_loss=0.44, loss=0.5


1661/2000 train loss: 0.4398111899693807  val loss: 0.7244891226291656


Epoch 1662/2000:  50%|▌| 3/6 [00:02<00:02,  1.03batch/s, avg_loss=0.541, loss=0.

In [ ]:
import os
weights_dir = "/projectnb/dunlop/chrisdc/gva_senior_design/image_processing/weights/"
name = '3-29-24_epochs2000'
weights_path = os.path.join(weights_dir, name + '.pkl')
experiment.save_experiment(weights_path)